In [1]:
from lxml import html  
import csv,os,json
import requests
from time import sleep
import ast
import re
from selenium import webdriver
import pandas as pd

In [26]:
driver = webdriver.Firefox(executable_path=r'C:\Geckodriver\geckodriver.exe')
driver.set_page_load_timeout(500)

In [27]:
def WalmartParser(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    driver.get(url)
    #print(driver)
    page = driver.page_source
    rpage = requests.get(url,headers=headers)
   
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page)
            script_data = doc.xpath('//script[contains(., "Specifications")]/text()')[0]
            
            XPATH_NAME = '//h1[contains(@class,"prod-ProductTitle")]/div/text()'
            XPATH_SALE_PRICE = '//span[@class="display-inline-block-xs prod-PaddingRight--s"]//text()'
            XPATH_CATEGORY = '//ol[@class="breadcrumb-list "]//text()'
            XPATH_RATING = '//span[@class="seo-avg-rating" and @itemprop="ratingValue"]/text()'
             
            
            ''' 
            XPATH_SALE_PRICE = '//span[contains(@class,"offer-price") and contains(@class,"a-size-medium")]//text()'
            #XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()' 
            XPATH_AUTHOR = '//a[contains(@class,"contributorNameID")]//text()'
            XPATH_ISBN10 = '//li/b[contains(text(),"ISBN-10:")]/following-sibling::text()'
            XPATH_PAGES = '//li/b[contains(text(),"Paperback")]/following-sibling::text()'
            XPATH_PUBLISHER = '//li/b[contains(text(),"Publisher")]/following-sibling::text()'
            XPATH_LANGUAGE = '//li/b[contains(text(),"Language")]/following-sibling::text()'
            XPATH_DIMENSIONS = '//li/b[contains(text(),"Dimensions")]/following-sibling::text()'
            XPATH_WEIGHT = '//li/b[contains(text(),"Weight")]/following-sibling::text()'
            XPATH_RATING = '//li/b[contains(text(),"Average Customer Review:")]/following-sibling::span/span/a/i/span/text()'
            '''
           
            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            RAW_RATING = doc.xpath(XPATH_RATING)
            print(RAW_SALE_PRICE)
           # RAW_AUTHOR = doc.xpath(XPATH_AUTHOR)
            '''
            
            
            ##RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)
            
            RAW_ISBN10 = doc.xpath(XPATH_ISBN10)
            RAW_PAGES = doc.xpath(XPATH_PAGES)
            RAW_PUBLISHER = doc.xpath(XPATH_PUBLISHER) 
            RAW_LANGUAGE = doc.xpath(XPATH_LANGUAGE)
            RAW_DIMENSIONS = doc.xpath(XPATH_DIMENSIONS)
            RAW_WEIGHT = doc.xpath(XPATH_WEIGHT)
            RAW_RATING = doc.xpath(XPATH_RATING)
            '''
            
            AUTHOR, PUBLISHER, LANGUAGE, PAGES, ISBN10, DIMENSIONS = None, None, None, None, None, None
            

            data = re.search(r'"Specifications":(.*?),"ShortDescription', script_data).group(1)
            #print(data)
            #import json
            #data = json.loads(data)
            data = ast.literal_eval(data)
            data = (data["specifications"]["values"][0])
            for item in data:
                #print item
                for k,v in item.items():
                    if k == "author":
                        RAW_AUTHOR = v['values']
                        AUTHOR = ''.join(RAW_AUTHOR).strip() if RAW_AUTHOR else None
                        
                    elif k == "publisher":
                        RAW_PUBLISHER = v['values']
                        PUBLISHER = ''.join(RAW_PUBLISHER).strip() if RAW_PUBLISHER else None

                    elif k == "languages":
                        RAW_LANGUAGE = v['values']
                        LANGUAGE = ''.join(RAW_LANGUAGE).strip() if RAW_LANGUAGE else None
                        
                    elif k == "number_of_pages":
                        RAW_PAGES = v['values']
                        PAGES = ''.join(RAW_PAGES).strip() if RAW_PAGES else None

                    elif k == "isbn-10":
                        RAW_ISBN10 = v['values']
                        ISBN10 = ''.join(RAW_ISBN10).strip() if RAW_ISBN10 else None
                    elif k == "dimension":
                        RAW_DIMENSIONS = v['values']
                        DIMENSIONS = ''.join(RAW_DIMENSIONS).strip() if RAW_DIMENSIONS else None
                    
            #AUTHOR = data
            
            #print AUTHOR
 
            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            
           
            
            #print AUTHOR, PAGES, PUBLISHER
            '''
            WEIGHT = ''.join(RAW_WEIGHT).strip().strip('()').strip() if RAW_WEIGHT else None
            RATING = ''.join(RAW_RATING).strip() if RAW_RATING else None
            #if not ORIGINAL_PRICE:
                #ORIGINAL_PRICE = SALE_PRICE
                '''
 
            if rpage.status_code!=200 or not NAME or NAME == "":
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                   # 'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    'AUTHOR' : AUTHOR,
                    'ISBN10' : ISBN10,
                    'PAGES' : PAGES,
                    'PUBLISHER' : PUBLISHER,
                    'LANGUAGE' : LANGUAGE,
                    'DIMENSIONS' : DIMENSIONS,
                    #'WEIGHT' : WEIGHT,
                    'RATING' : RATING,
                    }
 
            return data
        except Exception as e:
            print("Retrying with exception " + e)
            driver.get(url)
            #print(driver)
            rpage = requests.get(url,headers=headers)
            page = driver.page_source
 

In [28]:
extracted_data = []
def ReadAsin(AsinList):
    count = 0
    # AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    for i in AsinList:
        url = "https://www.walmart.com/ip/"+i
        print("Processing: "+url)
        extracted_data.append(WalmartParser(url))
        count += 1
        #sleep(5)
        print(count)
    f=open('data_walmart.json','w')
    json.dump(extracted_data,f,indent=4)
 

In [29]:
formatted_asin = []
df = pd.read_csv("walmart_product_asins.csv")
for index, row in df.iterrows():
    #print(row[1])
    formatted_asin.append(str(row[1]))

In [20]:
if __name__ == "__main__":
    ReadAsin(formatted_asin[1145:1500])

Processing: https://www.walmart.com/ip/23368505
['$', '8', '.', '79']
1
Processing: https://www.walmart.com/ip/25673860
['$', '7', '.', '99']
2
Processing: https://www.walmart.com/ip/24842896
['$', '16', '.', '19']
3
Processing: https://www.walmart.com/ip/161847750
['$', '14', '.', '69']
4
Processing: https://www.walmart.com/ip/40972530
['$', '4', '.', '49']
5
Processing: https://www.walmart.com/ip/25948252
['$', '7', '.', '99']
6
Processing: https://www.walmart.com/ip/404973
['$', '3', '.', '87']
7
Processing: https://www.walmart.com/ip/40569702
['$', '9', '.', '64']
8
Processing: https://www.walmart.com/ip/21049605
['$', '7', '.', '99']
9
Processing: https://www.walmart.com/ip/30036884
['$', '10', '.', '96']
10
Processing: https://www.walmart.com/ip/10599782
['$', '6', '.', '29']
11
Processing: https://www.walmart.com/ip/54134081
['$', '7', '.', '19']
12
Processing: https://www.walmart.com/ip/56174412
['$', '6', '.', '77']
13
Processing: https://www.walmart.com/ip/29270161
['$', '13'

TimeoutException: Message: Timeout loading page after 100000ms


In [23]:
print(len(extracted_data))

25
